In [ ]:
import geopandas as gpd
from shapely.geometry.multilinestring import MultiLineString

lines = gpd.read_file('water_lines.gpkg')

polygons = gpd.read_file('water_polygons.gpkg')



Из базовых идей:
1. Продлеваем реки немного и sjoin на пересечение с водой, фильтруем воду, геомку возвращаем.
2. Для больших озёр, не рек строим как у стива разбиение полигона - получаем точки и ещё раз вороного сверху, получаем озеро, теоретически разбитое ровненько по блокам одинаковым, в теории
3.

In [ ]:
from shapely import LineString
import math
import tifffile as tiff
import numpy as np
from skimage import measure
from skimage.filters import gaussian


in_path = "N59E030_FABDEM_V1-2.tif"
step_m = 3

def extend_linestring(line: LineString, distance: float = 1.0) -> LineString:
    if len(line.coords) < 2:
        raise ValueError("LineString must have at least two points.")

    x0, y0 = line.coords[0]
    x1, y1 = line.coords[1]
    dx_start = x0 - x1
    dy_start = y0 - y1
    length_start = math.hypot(dx_start, dy_start)
    unit_dx_start = dx_start / length_start
    unit_dy_start = dy_start / length_start
    new_start = (x0 + unit_dx_start * distance, y0 + unit_dy_start * distance)

    xN_1, yN_1 = line.coords[-2]
    xN, yN = line.coords[-1]
    dx_end = xN - xN_1
    dy_end = yN - yN_1
    length_end = math.hypot(dx_end, dy_end)
    unit_dx_end = dx_end / length_end
    unit_dy_end = dy_end / length_end
    new_end = (xN + unit_dx_end * distance, yN + unit_dy_end * distance)

    new_coords = [new_start] + list(line.coords[1:-1]) + [new_end]
    return LineString(new_coords)

with tiff.TiffFile(in_path) as tif:
    page = tif.pages[0]
    tags = page.tags
    scale = tags.get("ModelPixelScaleTag")
    tie = tags.get("ModelTiepointTag")
    if scale is None or tie is None:
        raise ValueError("Не найдены GeoTIFF-теги ModelPixelScaleTag / ModelTiepointTag")

    sx, sy = float(scale.value[0]), float(scale.value[1])
    i0, j0, _, X0, Y0, _ = tie.value[:6]
    i0, j0, X0, Y0 = float(i0), float(j0), float(X0), float(Y0)

    width = page.imagewidth
    height = page.imagelength

    hx = sx * 0.5
    hy = sy * 0.5
    
    x_min = X0 - hx
    x_max = X0 + (width - 0.5) * sx
    y_max = Y0 + hy
    y_min = Y0 - (height - 0.5) * sy

    arr = page.asarray()
    if arr.ndim == 3:
        arr = arr[0]

    nd_tag = page.tags.get("GDAL_NODATA")
    nodata = float(nd_tag.value) if nd_tag is not None else -9999.0

arr = np.where(arr == nodata, np.nan, arr).astype(float)

zmin = np.nanmin(arr)
zmax = np.nanmax(arr)

z_start = math.floor(zmin / step_m) * step_m
z_stop = math.ceil(zmax / step_m) * step_m
levels = np.arange(z_start, z_stop + step_m, step_m)

arr = np.round(arr/step_m)*step_m

lines = []
lvls = []
for lev in levels:
    contours = measure.find_contours(arr, level=lev)
    for cnt in contours:
        rows = cnt[:, 0]
        cols = cnt[:, 1]
        X = x_min + (cols - i0) * sx
        Y = y_max - (rows - j0) * sy

        coords = list(zip(X, Y))
        if len(coords) < 2:
            continue
        line = LineString(coords)
        if line.is_empty:
            continue
        if not line.is_closed:
            line = extend_linestring(line, 0.001)
            
        lines.append(line)
        lvls.append(lev)

In [ ]:
from shapely.ops import polygonize, unary_union
from shapely import build_area, node, MultiLineString, Polygon
import geopandas as gpd

lines_w_bounds = lines + [LineString(Polygon.from_bounds(x_min, y_min, x_max, y_max).exterior)]


gpd.GeoDataFrame(geometry=list(polygonize(node(MultiLineString(lines_w_bounds)))), crs=4326).to_file('test3.gpkg')

In [ ]:
from shapely.ops import polygonize, unary_union
from shapely import build_area, node, MultiLineString, Polygon
import geopandas as gpd

lines_w_bounds = lines + [LineString(Polygon.from_bounds(x_min, y_min, x_max, y_max).exterior)]

polygons_gdf = gpd.GeoDataFrame(geometry=list(polygonize(node(MultiLineString(lines_w_bounds)))), crs=4326)
# lines_gdf = gpd.GeoDataFrame(lvls, geometry=lines, crs=4326, columns=['level'])


In [ ]:
rep_pts = polygons_gdf.representative_point()
col = np.round((rep_pts.x.values - x_min) / sx + i0).astype(int)
row = np.round((y_max -  rep_pts.y.values) / sy + j0).astype(int)

h, w = arr.shape
col = np.clip(col, 0, w - 1)
row = np.clip(row, 0, h - 1)

vals = arr[row, col]

polygons_gdf["level"] = vals

In [ ]:
polygons_gdf.to_file('test.gpkg')

In [ ]:
gpd.GeoDataFrame(lvls, geometry=lines, crs=4326, columns=['level']).to_file('test_lines_1m.gpkg')